# Analysis

In [1]:
import pandas as pd
pd.set_option("display.max_columns",100)

In [2]:
data = pd.read_csv('../data/train_set.csv')

In [4]:
data.shape

(9566, 45)

## Train simple model to test API

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X = data.copy()
y = X.pop('label')

In [ ]:
model = LogisticRegression()
model.fit(X[['user_age', 'body_length', 'channels']], y)

In [ ]:
X[['user_age', 'body_length', 'channels']].dtypes

## Extensive preprocessing

In [30]:
X = data.copy()
y = X.pop('label')

In [34]:
X.head()

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,sample_uuid
0,1351796400,2796,8,US,USD,1.0,"<p><em><strong><span style=""font-size: medium;...",yahoo.com,1349123276,1351364400,1.349129e+09,1351351800,0,113.40,0,NaN,1,y,Traffic Safety Merit Badge - Class 2012-10-27-...,76,9,219,4481350,<p>Scoutmaster Bucky provides advancement oppo...,16.0,Scoutmaster Bucky,0.0,NaN,ACH,"[{u'name': u'Brian Reiners', u'created': u'201...",26.0,26,1,"[{u'event_id': 4481350, u'cost': 11.0, u'avail...",1116,1252729123,3,317 2nd Ave NW,US,45.119583,-93.405612,Osseo High School - 2012 Fall University of Sc...,MN,189dba2c-5a40-455f-9d23-ad2562f16d65
1,1354600800,9900,5,US,USD,0.0,"<p class=""MsoNormal"" style=""margin-bottom: .00...",miamishortfilmfestival.com,1349752111,1354168800,1.352340e+09,1354163400,0,144.00,0,NaN,1,y,All About the Music - Paragon Grove 13,38,8,10,4542210,"<p style=""margin: 0px; font-style: normal; fon...",22.0,Miami short Film Festival,16.0,NaN,ACH,"[{u'name': u'', u'created': u'2011-11-24 03:12...",21.0,51,1,"[{u'event_id': 4542210, u'cost': 10.35, u'avai...",346,1319812769,3,"3015 Grand Ave # 322 Coconut Grove,",US,25.728702,-80.241981,Paragon Grove 13,FL,55610345-19c0-4439-a099-0557a57b7891
2,1353391200,65381,5,US,USD,1.0,"<p style=""text-align: center;""> </p>\r\n<div s...",LDJCapital.com,1349360623,1352959200,1.349365e+09,1352948400,1,2080.25,0,0.0,1,y,United Nations Crowdfunding Conference for Afr...,82,1,37,4509050,"<div style=""color: #000000; font-family: arial...",11.0,www.TheSohoLoft.com,11.0,NaN,ACH,"[{u'name': u'', u'created': u'2012-01-06 03:12...",41.0,41,1,"[{u'event_id': 4509050, u'cost': 195.0, u'avai...",280,1325195187,3,405 E 42nd St,US,40.748808,-73.969707,"United Nations Headquarters, New York, NY 10017",NY,54eebf8c-ce29-4f9b-a45e-064a0b8d2794
3,1313964000,9536,11,US,USD,1.0,"<p><span class=""vevent""></span></p>\r\n<p clas...",stemforward.org,1302790002,1313532000,1.302809e+09,1313503200,0,3550.00,0,0.0,1,y,STEM Forward 6th Annual Scholarship Golf Outing,47,12,4,1523019,"<p style=""text-align: left;""><font size=""2""><a...",0.0,STEM Forward,11.0,NaN,ACH,"[{u'name': u'', u'created': u'2010-06-28 01:11...",124.0,124,0,"[{u'event_id': 1523019, u'cost': 94.01, u'avai...",609,1250178369,3,S100 W14020 Hwy 36,US,42.858041,-88.101070,Muskego Lakes Country Club,WI,b08dda95-9b73-4dcf-8914-9d2b058e6d96
4,1367362800,9080,8,US,USD,1.0,"<p class=""MsoNormal"" style=""font-family: 'Helv...",puppetlabs.com,1360879873,1366930800,1.360880e+09,1366732800,0,30177.00,1,0.0,1,y,Puppet Fundamentals Training: London_PAID 0501...,65,8,203,5521418,NaN,10.0,Puppet Labs,10.0,NaN,ACH,"[{u'name': u'', u'created': u'2010-12-14 01:13...",68.0,68,1,"[{u'event_id': 5521418, u'cost': 2313.2, u'ava...",961,1277843277,3,24 Eversholt Street,GB,51.528448,-0.132030,Euston House,London,5a35050a-5e10-4df2-b79e-c4e17b6c9108


In [38]:
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [45]:
categorical_var = ['payout_type', 'venue_country', 'venue_state', 'email_domain']
int_vars = X.dtypes.loc[X.dtypes == 'int64'].index.tolist()
num_vars = X.dtypes.loc[X.dtypes == 'float64'].index.tolist()

In [42]:
X.shape

(9566, 44)

In [53]:
X_clean = X[int_vars] # categorical_var + + num_vars

In [47]:
X_clean.shape

(9566, 31)

In [43]:
encoder = ce.OneHotEncoder(cols=categorical_var, 
                           impute_missing=False, handle_unknown='ignore')
X_one_hot = encoder.fit_transform(X)

In [44]:
X_one_hot.shape

(9566, 4675)

## Grid Search

Now that we have the API working, let's test a more complicated model.

In [49]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [54]:
param_grid = {"n_estimators" : [10, 20, 30],

             "max_depth" : [None, 6, 8, 10],

             "max_leaf_nodes": [None, 5, 10, 20],

             "min_impurity_decrease": [0.1, 0.2, 0.3]}

rf_clf = GridSearchCV(RandomForestClassifier(), param_grid, scoring='neg_log_loss', n_jobs=-1, verbose=2, cv=5)

rf_clf = rf_clf.fit(X_clean, y)

grid_size = np.product([len(value) for value in param_grid.values()])
print("grid size: {}".format(grid_size))
print("best score: {}".format(rf_clf.best_score_))
print("best estimator: {}".format(rf_clf.best_estimator_))



Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 -   0.1s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 -   0.1s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.1, n_estimators=10 -   0.1s
[CV] max_depth=None, max_leaf_nodes=None, min_impu

[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.3, n_estimators=30 -   0.1s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.3, n_estimators=30 
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=30 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=20 -   0.1s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=20 -   0.1s
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=10 -   0.1s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=20 
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.2, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.2, n_estimators=30 -   0.1s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_decrease=0.2, n_estimators=30 
[CV]  max_depth=None, max_leaf

[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.0s


[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.2, n_estimators=30 -   0.1s
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=30 -   0.1s
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=10 -   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.2, n_estimators=30 
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=30 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=10 -   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=20 -   0.1s
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=10 
[CV] max_depth=None, max_lea

[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=30 
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=30 
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=30 -   0.1s
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=30 
[CV]  max_depth=None, max_leaf_nodes=20, min_impurity_decrease=0.1, n_estimators=10 -   0.1s
[CV] max_depth=None, max_leaf_nodes=20, min_impurity_decrease=0.1, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=30 -   0.1s
[CV] max_depth=None, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=30 
[CV]  max_depth=None, max_leaf_

...

[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=10 -   0.0s
[CV] max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=10 
[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=30 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=None, min_impurity_decrease=0.3, n_estimators=30 -   0.2s
[CV] max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.1, n_estimators=30 
[CV] max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=30 
[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.2, n_estimators=20 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=10 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.1, n_estimators=10 
[CV]  max_depth=10, max_leaf_nodes=5, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=5, min_i

[CV]  max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.1, n_estimators=10 
[CV]  max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=30 -   0.2s
[CV] max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=30 
[CV]  max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=30 
[CV]  max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.2, n_estimators=20 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=10, min_impurity_decrease=0.3, n_estimators=20 
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.1, n_estimators=20 
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_d

[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.2, n_estimators=20 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=10 -   0.0s
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=10 
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.2, n_estimators=30 -   0.1s
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.2, n_estimators=30 
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=20 
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=10 -   0.0s
[CV] max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=10 
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=20 -   0.1s
[CV]  max_depth=10, max_leaf_nodes=20, min_impurity_decrease=0.3, n_estimators=10 -   0.0s
[CV]  max_depth=10, max_leaf_nodes=

[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   10.2s finished


NameError: name 'np' is not defined

In [55]:
rf_clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=10,
            min_impurity_decrease=0.3, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
rf_clf.best_params_

{'max_depth': 10,
 'max_leaf_nodes': 10,
 'min_impurity_decrease': 0.3,
 'n_estimators': 20}

## Train and save final model

In [60]:
best_model = RandomForestClassifier(**rf_clf.best_params_)

In [61]:
best_model.fit(X_clean, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=10,
            min_impurity_decrease=0.3, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Saving best model as pickle

In [62]:
from sklearn.externals import joblib
joblib.dump(best_model, '../api/best_model.pkl') # other folder eventually

['../api/best_model.pkl']

In [ ]:
clf = joblib.load('model.pkl') 

In [ ]:
testcase = pd.DataFrame(X.loc[0, ['user_age', 'body_length', 'channels']]).T

In [ ]:
clf.predict(testcase)

In [ ]:
clf.predict_proba(testcase)[0, 1]